In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, RFE

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [19]:
data = pd.read_csv('./modelling_data/concatenated_data/cleaned_data/all_games_1819_2324_cleaned.csv')

In [20]:
data.shape

(36169, 162)

In [21]:
data.groupby(['league', 'season']).apply(lambda x: x['game_week'].isna().sum())

league              season
belgium pro league  18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
bundesliga          18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
czech first league  18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
denmark             18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
ekstraklasa         18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
epl champions

In [22]:
data.isna().sum()

attendance                                                  0
home_team_name                                              0
away_team_name                                              0
game_week                                                   0
home_team_goal_count                                        0
away_team_goal_count                                        0
total_goal_count                                            0
total_goals_at_half_time                                    0
home_team_goal_count_half_time                              0
away_team_goal_count_half_time                              0
home_team_corner_count                                      0
away_team_corner_count                                      0
home_team_yellow_cards                                      0
home_team_red_cards                                         0
away_team_yellow_cards                                      0
away_team_red_cards                                         0
home_tea

In [23]:
data.dtypes

attendance                                                  float64
home_team_name                                               object
away_team_name                                               object
game_week                                                   float64
home_team_goal_count                                          int64
away_team_goal_count                                          int64
total_goal_count                                              int64
total_goals_at_half_time                                      int64
home_team_goal_count_half_time                                int64
away_team_goal_count_half_time                                int64
home_team_corner_count                                        int64
away_team_corner_count                                        int64
home_team_yellow_cards                                        int64
home_team_red_cards                                           int64
away_team_yellow_cards                          